In [93]:
%reload_ext autoreload
%autoreload 2

import pandas as pd

In [94]:
from pathlib import Path

from model import (
    extract_emp_data,
    extract_trip_data,
    extract_tour_data,
)
from totals import summarize_totals
from rates import summarize_rates

In [95]:
dir_ = Path('T:/data/base_2022/CVM/Calib_CVM_B/')

In [80]:
emp = extract_emp_data(path=dir_/'model_output/Zonal Properties CVM.csv')
emp

,emp
industry,
Industry,207356
Wholesale,43668
Retail,144185
Government\Office,494292
Service,572051
Transport,37037
Fleet Allocator,0


In [81]:
tours = extract_tour_data(path=dir_/'model_output/commercialVehicleTours.csv')
tours.head(2)

,industry,mode
tour_id,,
1,Industry,Light Heavy Duty Truck
2,Industry,Drive Alone


In [89]:
tours.industry.unique()

['Industry', 'Government\Office', 'Fleet Allocator', 'Service', 'Transport', 'Retail', 'Wholesale']
Categories (7, object): ['Industry', 'Wholesale', 'Retail', 'Government\Office', 'Service', 'Transport', 'Fleet Allocator']

In [91]:
tours['mode'].unique()

array(['Light Heavy Duty Truck', 'Drive Alone', 'Heavy Heavy Duty Truck',
       'Medium Heavy Duty Truck'], dtype=object)

In [82]:
trips = extract_trip_data(path=dir_/'model_output/commercialVehicleTrips.csv')

# add industry and tour mode (tour-level data assignment) to trip data
trips = trips.reset_index().merge(
    tours,
    how='left',
    on='tour_id'
).set_index('trip_id')

trips.head(2)

,tour_id,stop_id,weight_trip,weight_person,distance,industry,mode
trip_id,,,,,,,
1,1,1,3.36,3.36,13.258280,Industry,Light Heavy Duty Truck
2,1,2,3.36,3.36,13.258282,Industry,Light Heavy Duty Truck


In [87]:
totals = summarize_totals(emp=emp, tours=tours, trips=trips)
totals.to_csv(dir_/'model_results_totals.csv')
totals

,emp,tours,trips,vmt,med_heavy_vmt
industry,,,,,
Industry,207356,44000,161734,1.281649e+06,262916.438097
Wholesale,43668,8810,52166,5.315402e+05,48038.489872
Retail,144185,30836,169507,1.218760e+06,63974.052291
Government\Office,494292,67290,264547,2.036895e+06,139902.485356
Service,572051,66280,262627,1.752031e+06,112414.880950
Transport,37037,10375,41896,2.636991e+05,67910.684108
Fleet Allocator,0,57400,459300,3.827122e+06,952756.273795


In [88]:
rates = summarize_rates(trips=trips, totals=totals)
rates.to_csv(dir_/'model_results_rates.csv')
rates

,tours_per_emp,trips_per_tour,avg_trip_len,pct_light
industry,,,,
Industry,0.212195,3.675773,7.924425,0.794861
Wholesale,0.201750,5.921226,10.189399,0.909624
Retail,0.213864,5.497049,7.190030,0.947509
Government\Office,0.136134,3.931446,7.699559,0.931316
Service,0.115864,3.962387,6.671175,0.935837
Transport,0.280125,4.038169,6.294134,0.742469
Fleet Allocator,inf,8.001742,8.332510,0.751052
